In [ ]:
import os
import csv
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import Sequence
import matplotlib.pyplot as plt

class DataGenerator(Sequence):
   
    def __init__(self, file_paths, labels, batch_size, img_size):
        self.file_paths = file_paths
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size

    def __len__(self):
        return int(np.ceil(len(self.file_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_paths = self.file_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]

        images = []
        for path in batch_paths:
            image = cv2.imread(path)
            if image is not None:
                image = cv2.resize(image, self.img_size)
                images.append(image / 255.0)  # Normalize to [0, 1]

        return np.array(images), np.array(batch_labels)

def load_file_paths_and_labels(dataset_path):
    """
    Load file paths and age labels without loading images into memory.
    """
    file_paths = []
    labels = []

    for file in os.listdir(dataset_path):
        if file.lower().endswith((".jpg", ".jpeg", ".png")):
            try:
                age = int(file.split("_")[0])  # Extract age from file name
                file_paths.append(os.path.join(dataset_path, file))
                labels.append(age)
            except Exception as e:
                print(f"Error processing file {file}: {e}")
                continue

    if not file_paths or not labels:
        raise ValueError("Dataset is empty or invalid.")

    return file_paths, labels

def predict_and_save_to_csv(model, dataset_path, csv_file_path, img_size):
    """
    Predict the age for all images in the dataset and save results to a CSV file.
    """
    print("Predicting ages and saving results to CSV...")
    with open(csv_file_path, mode="w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["File Name", "Predicted Age"])  # Header

        for file in os.listdir(dataset_path):
            if file.lower().endswith((".jpg", ".jpeg", ".png")):
                try:
                    file_path = os.path.join(dataset_path, file)
                    image = cv2.imread(file_path)
                    if image is None:
                        print(f"Skipping file: {file} (unable to load)")
                        continue

                    # Preprocess the image
                    image = cv2.resize(image, img_size)
                    image = np.expand_dims(image / 255.0, axis=0)  # Normalize and add batch dimension

                    # Predict age
                    predicted_age = model.predict(image, verbose=0)[0][0]
                    predicted_age = round(predicted_age, 2)

                    # Save to CSV
                    writer.writerow([file, predicted_age])
                    print(f"Processed: {file}, Predicted Age: {predicted_age}")

                except Exception as e:
                    print(f"Error processing file {file}: {e}")

    print(f"CSV file created at: {csv_file_path}")

# Paths and settings
dataset_path = "C:/Users/Admin/OneDrive/Desktop/Null class/age_detection/datasets/utkface_aligned_cropped/UTKFace" #Specify the dataset path where the data is stored
csv_file_path = "age_detection.csv"
img_size = (128, 128)
batch_size = 32

# Load file paths and labels
print("Loading file paths and labels...")
file_paths, labels = load_file_paths_and_labels(dataset_path)
print(f"Loaded {len(file_paths)} images with labels.")

# Split the dataset
file_paths_train, file_paths_test, labels_train, labels_test = train_test_split(
    file_paths, labels, test_size=0.2, random_state=42)

train_gen = DataGenerator(file_paths_train, labels_train, batch_size, img_size)
test_gen = DataGenerator(file_paths_test, labels_test, batch_size, img_size)

# Load and fine-tune the model
print("Loading pre-trained model...")
base_model = MobileNetV2(include_top=False, input_shape=(img_size[0], img_size[1], 3), weights="imagenet")

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(1, activation="linear")(x)  # Regression for age prediction
model = Model(inputs=base_model.input, outputs=output)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss="mean_squared_error", metrics=["mae"])

# Train the model
print("Training the model...")
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-5)

history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=50,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model
print("Evaluating the model...")
test_loss, test_mae = model.evaluate(test_gen, verbose=2)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

# Save the trained model
model_save_path = "C:/Users/Admin/OneDrive/Desktop/Null class/age_detection/saved_model" #Specify the path where want to save model
model.save(model_save_path)
print(f"Model saved at {model_save_path}")

# Predict and save results to CSV
predict_and_save_to_csv(model, dataset_path, csv_file_path, img_size)

# Plot training history
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()
